## 預測價格跟區間

In [7]:
import os
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from joblib import Parallel, delayed

# --------------------------
# 1. 讀取資料
# --------------------------
df = pd.read_csv('/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv')

# --------------------------
# 2. 欄位定義
# --------------------------
cat_cols  = ['出發時段','出發機場代號','抵達時段','航空聯盟','機型分類','假期','是否為平日']
group_col = '抵達機場代號'
num_cols  = ['飛行時間_分鐘','經濟指標','機場指標','competing_flights']
log_col   = '平均價格_log'
true_col  = '平均價格'

# --------------------------
# 3. 分艙等並標記真實價格
# --------------------------
eco_raw = df[df['艙等']=='經濟艙'][cat_cols + [group_col] + num_cols + [log_col] + [true_col]].copy()
biz_raw = df[df['艙等']=='商務艙'][cat_cols + [group_col] + num_cols + [log_col] + [true_col]].copy()

# --------------------------
# 4. 編碼與標準化
# --------------------------
def encode_and_scale(df_raw):
    df_enc = pd.get_dummies(df_raw, columns=cat_cols, drop_first=True)
    scaler = StandardScaler()
    df_enc[num_cols] = scaler.fit_transform(df_enc[num_cols])
    return df_enc, scaler

eco_enc, eco_scaler = encode_and_scale(eco_raw)
biz_enc, biz_scaler = encode_and_scale(biz_raw)

# --------------------------
# 5. 取共同特徵欄位
# --------------------------
common_feats = list(set(eco_enc.columns) & set(biz_enc.columns))
for col in [log_col, true_col, group_col]:
    if col in common_feats:
        common_feats.remove(col)

# --------------------------
# 6. 訓練模型 & 拆分測試集
# --------------------------
def split_and_train(df_enc, model_type='xgb', seed=123):
    X = df_enc[common_feats]
    y = df_enc[log_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    if model_type == 'xgb':
        model = XGBRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
    else:
        model = RandomForestRegressor(n_estimators=100, random_state=seed, n_jobs=-1)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"🔹[{model_type}] 測試集 MSE = {mean_squared_error(y_test, y_pred):.4f}, R2 = {r2_score(y_test, y_pred):.4f}")
    
    return model, X_train, y_train, X_test

eco_model, eco_X_train, eco_y_train, eco_X_test = split_and_train(eco_enc, model_type='xgb')
biz_model, biz_X_train, biz_y_train, biz_X_test = split_and_train(biz_enc, model_type='rf')

# --------------------------
# 7. Bootstrap 單次訓練
# --------------------------
def train_and_predict_bootstrap(i, X_train, y_train, X_test, model_type):
    boot_idx = np.random.choice(len(X_train), len(X_train), replace=True)
    X_boot = X_train.iloc[boot_idx]
    y_boot = y_train.iloc[boot_idx]

    if model_type == 'xgb':
        model = XGBRegressor(n_estimators=100, random_state=123+i, n_jobs=-1)
    else:
        model = RandomForestRegressor(n_estimators=100, random_state=123+i, n_jobs=-1)

    model.fit(X_boot, y_boot)
    preds = model.predict(X_test)
    return preds

# --------------------------
# 8. 預測價格 + 預測區間並存在一起
# --------------------------
def predict_and_save_testset_with_interval(raw_df, model, X_train, y_train, X_test, scaler, prefix, model_type='xgb', n_bootstrap=100):
    out_dir = 'predict_data/short'
    os.makedirs(out_dir, exist_ok=True)

    test_index = X_test.index
    test_df = raw_df.loc[test_index]

    # 計算 sigma²（測試集誤差變異數）
    log_preds_test = model.predict(X_test)
    sigma2 = np.var(log_preds_test - np.log1p(test_df[true_col]))

    for code in sorted(test_df[group_col].unique()):
        combos = test_df[test_df[group_col] == code][cat_cols + num_cols + [true_col]].drop_duplicates()
        if combos.empty:
            continue

        # 處理 combo 特徵
        enc = pd.get_dummies(combos, columns=cat_cols, drop_first=True)
        enc = enc.reindex(columns=common_feats, fill_value=0)
        enc[num_cols] = scaler.transform(combos[num_cols])

        # 預測平均價格
        log_preds = model.predict(enc)
        combos['預測_平均價格'] = np.expm1(log_preds + 0.5 * sigma2)

        # Bootstrap 預測區間
        results = Parallel(n_jobs=-1)(
            delayed(train_and_predict_bootstrap)(i, X_train, y_train, enc, model_type)
            for i in range(n_bootstrap)
        )
        y_preds_bootstrap = np.array(results)  # (n_bootstrap, n_samples)

        # 預測中位數、上下界
        preds_median = np.expm1(np.median(y_preds_bootstrap, axis=0))
        preds_lower = np.expm1(np.percentile(y_preds_bootstrap, 2.5, axis=0))
        preds_upper = np.expm1(np.percentile(y_preds_bootstrap, 97.5, axis=0))

        # 加上預測區間
        combos['預測中位數'] = preds_median
        combos['預測區間下界'] = preds_lower
        combos['預測區間上界'] = preds_upper
        combos[group_col] = code

        # 儲存
        fn = f'{out_dir}/{prefix}_{code}.csv'
        combos.to_csv(fn, index=False)
        print(f"✅ 預測結果已存：{fn}（共 {len(combos)} 筆）")

# --------------------------
# 9. 執行預測與儲存
# --------------------------
predict_and_save_testset_with_interval(eco_raw, eco_model, eco_X_train, eco_y_train, eco_X_test, eco_scaler, prefix='eco', model_type='xgb', n_bootstrap=100)
predict_and_save_testset_with_interval(biz_raw, biz_model, biz_X_train, biz_y_train, biz_X_test, biz_scaler, prefix='biz', model_type='rf', n_bootstrap=100)

🔹[xgb] 測試集 MSE = 0.0546, R2 = 0.8535
🔹[rf] 測試集 MSE = 0.0092, R2 = 0.8909
✅ 預測結果已存：predict_data/short/eco_BKK.csv（共 298 筆）
✅ 預測結果已存：predict_data/short/eco_GMP.csv（共 31 筆）
✅ 預測結果已存：predict_data/short/eco_HKG.csv（共 705 筆）
✅ 預測結果已存：predict_data/short/eco_HND.csv（共 184 筆）
✅ 預測結果已存：predict_data/short/eco_ICN.csv（共 194 筆）
✅ 預測結果已存：predict_data/short/eco_NRT.csv（共 384 筆）
✅ 預測結果已存：predict_data/short/eco_SIN.csv（共 193 筆）
✅ 預測結果已存：predict_data/short/biz_BKK.csv（共 181 筆）
✅ 預測結果已存：predict_data/short/biz_GMP.csv（共 12 筆）
✅ 預測結果已存：predict_data/short/biz_HKG.csv（共 279 筆）
✅ 預測結果已存：predict_data/short/biz_HND.csv（共 136 筆）
✅ 預測結果已存：predict_data/short/biz_ICN.csv（共 146 筆）
✅ 預測結果已存：predict_data/short/biz_NRT.csv（共 218 筆）
✅ 預測結果已存：predict_data/short/biz_SIN.csv（共 118 筆）
